Importamos las librerias necesarias para el preprocesado y entrenamiento del modelo

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import time

Cargamos las bases de datos Train y Test descargadas de Kaggle

In [4]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')

Analizamos un poco el test y el train

In [5]:

train.describe()

,ID,PERIODO
count,692500.000000,692500.000000
mean,494606.130576,20198.366679
std,285585.209455,10.535037
min,1.000000,20183.000000
25%,247324.750000,20195.000000
50%,494564.500000,20195.000000
75%,741782.500000,20203.000000
max,989286.000000,20213.000000


In [6]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [7]:
test.describe()

,ID,PERIODO
count,296786.000000,296786.000000
mean,494730.695238,20198.379078
std,285576.351746,10.525743
min,2.000000,20183.000000
25%,247318.250000,20195.000000
50%,494808.500000,20195.000000
75%,742441.000000,20203.000000
max,989285.000000,20213.000000


In [8]:
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa


Guardamos el rendimiento y el id de la base de datos train para mas adelante.
Y luego se le quitó a train el rendimiento para poder concatenarlo con el test.

In [9]:
train_labels = train[['ID', 'RENDIMIENTO_GLOBAL']]

# Eliminar la columna label de train para unirlo con test
train = train.drop(columns=['RENDIMIENTO_GLOBAL'])

# Unir train y test
datos_combinados = pd.concat([train, test], ignore_index=True)

Miramos un poco los datos combinados para ver si quedaron con la cantidad de filas combinadas

In [10]:
datos_combinados.describe()

,ID,PERIODO
count,989286.00000,989286.000000
mean,494643.50000,20198.370398
std,285582.41354,10.532246
min,1.00000,20183.000000
25%,247322.25000,20195.000000
50%,494643.50000,20195.000000
75%,741964.75000,20203.000000
max,989286.00000,20213.000000


In [11]:
datos_combinados.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa


En el analisis se habia visto que la columna FAMI_TIENEINTERNET.1 estaba repetida y que ESTU_PRIVADO_LIBERTAD estaba desbalanceada por lo que se eliminaron.

In [12]:
datos_combinados.drop(columns=['ESTU_PRIVADO_LIBERTAD'], inplace=True)
datos_combinados.drop(columns=['FAMI_TIENEINTERNET.1'], inplace=True)

Cambiar las columnas que solo tenian Si y No por 1 y 0

In [13]:
yes_no= {'Si': 1, 'No': 0}

columnas_a_convertir = ['FAMI_TIENECOMPUTADOR', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENEAUTOMOVIL',
                      'FAMI_TIENELAVADORA', 'FAMI_TIENEINTERNET']

for column in columnas_a_convertir:
    datos_combinados[column] = datos_combinados[column].map(yes_no)

datos_combinados.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,1.0,Técnica o tecnológica incompleta,1.0,1.0,0.0,1.0,Postgrado
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,0.0,Técnica o tecnológica completa,1.0,0.0,0.0,1.0,Técnica o tecnológica incompleta
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,1.0,Secundaria (Bachillerato) completa,1.0,0.0,0.0,0.0,Secundaria (Bachillerato) completa
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,1.0,No sabe,1.0,0.0,0.0,1.0,Secundaria (Bachillerato) completa
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,1.0,Primaria completa,1.0,1.0,0.0,1.0,Primaria completa


Revisamos las columnas con valores nulos

In [14]:
null_counts = datos_combinados.isnull().sum()
null_counts = null_counts[null_counts > 0]

null_counts

ESTU_VALORMATRICULAUNIVERSIDAD     9010
ESTU_HORASSEMANATRABAJA           44236
FAMI_ESTRATOVIVIENDA              45932
FAMI_TIENEINTERNET                38168
FAMI_EDUCACIONPADRE               33171
FAMI_TIENELAVADORA                57032
FAMI_TIENEAUTOMOVIL               62541
ESTU_PAGOMATRICULAPROPIO           9305
FAMI_TIENECOMPUTADOR              54542
FAMI_EDUCACIONMADRE               33887
dtype: int64

Reemplazamos los valores nulos de las variables categoricas por la moda

In [15]:
for column in null_counts.index:
    mode_value = datos_combinados[column].mode()[0]
    datos_combinados[column].fillna(mode_value, inplace=True)

remaining_nulls = datos_combinados.isnull().sum().sum()
remaining_nulls


C:\Users\joseo\AppData\Local\Temp\ipykernel_12856\4204642009.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos_combinados[column].fillna(mode_value, inplace=True)


0

Se realiza depuracion para la variable ESTU_PRGM_ACADEMICO agrupando por programas que tuvieran errores ortograficos, similitud en su nombre o que fueran la misma y estuvieran nombradas de diferente manera.

Se hizo la funcion normalizar_carrera la cual busca el programa con cierta similitud de palabras y la agrupa en un nuevo nombre

In [17]:
def normalizar_carrera(nombre):
    if 'ADMINISTRACIÓN TÚRÍSTICA'in nombre:
        return'ADMINISTRACION DE EMPRESAS TURISTICA'
    return nombre
datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].apply(normalizar_carrera)

Por temas de no poder trabajar en un mismo entorno, diferentes horarios, reinicios de kernel se dividio el trabajo y se iba guardando la base de datos depurada para luego seguir en este mismo notebook.

In [17]:
datos_combinados = pd.read_csv('datos_mezlados.csv')

----------------------------------------------------------------

Se le aplica one hot a las variables categoricas y como se puede observar la base de datos queda con 752 columnas comparado a como quedaria con el original que es 1033, se depuraron 281 programas.

In [18]:
categorical_columns = datos_combinados.select_dtypes(include=['object']).columns

datos_combinados = pd.get_dummies(datos_combinados, columns=categorical_columns, drop_first=True)

datos_combinados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989286 entries, 0 to 989285
Columns: 752 entries, ID to FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
dtypes: bool(745), float64(5), int64(2)
memory usage: 755.7 MB


Como one hot lo deja en TRUE y FALSE se reemplaza por 1 y 0

In [19]:
for col in datos_combinados.columns:
    if datos_combinados[col].dtype == 'bool':
        datos_combinados[col] = datos_combinados[col].astype(int)

In [20]:
datos_combinados.head()

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,904256,20212,1.0,1.0,1.0,0.0,1.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,645256,20212,0.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,308367,20203,1.0,1.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,470353,20195,1.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,989032,20212,1.0,1.0,1.0,0.0,1.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Se separan las bases de datos de test y train respecto a los ID's que habiamos separados al principio para que queden listos para entrenar y probar.

In [21]:
train_listos = datos_combinados[datos_combinados['ID'].isin(train_labels['ID'])]
test_listos = datos_combinados[~datos_combinados['ID'].isin(train_labels['ID'])]


train_listos = train_listos.merge(train_labels, on='ID', how='left')

Y como se observa quedaron con el mismo numero de filas iniciales.

In [22]:
train_listos.describe()

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.0,...,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,20198.366679,0.894069,0.870994,0.341186,0.438702,0.918084,0.000114,0.000169,0.0,...,0.020914,0.002770,0.004357,0.066781,0.081047,0.143567,0.238856,0.116985,0.129303,0.039759
std,285585.209455,10.535037,0.307749,0.335207,0.474108,0.496229,0.274237,0.010680,0.012997,0.0,...,0.143097,0.052555,0.065861,0.249643,0.272907,0.350650,0.426385,0.321402,0.335535,0.195392
min,1.000000,20183.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,20195.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,494564.500000,20195.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,741782.500000,20203.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,989286.000000,20213.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
test_listos.describe()

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
count,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,...,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000,296786.000000
mean,494730.695238,20198.379078,0.894968,0.872090,0.341131,0.438397,0.917951,0.000098,0.000152,0.000003,...,0.021345,0.002645,0.004333,0.067554,0.082211,0.142018,0.238798,0.116468,0.129962,0.039254
std,285576.351746,10.525743,0.306595,0.333991,0.474090,0.496191,0.274440,0.009885,0.012313,0.001836,...,0.144533,0.051362,0.065684,0.250979,0.274686,0.349069,0.426350,0.320786,0.336263,0.194199
min,2.000000,20183.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,247318.250000,20195.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,494808.500000,20195.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,742441.000000,20203.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,989285.000000,20213.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Transformamos las 4 categorias de rendimiento a numeros.

In [24]:
label_encoder = LabelEncoder()
train_listos['RENDIMIENTO_GLOBAL'] = label_encoder.fit_transform(train_listos['RENDIMIENTO_GLOBAL'])

In [25]:
train_listos.head()

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,...,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL
0,904256,20212,1.0,1.0,1.0,0.0,1.0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,645256,20212,0.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,308367,20203,1.0,1.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,470353,20195,1.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,1.0,1.0,1.0,0.0,1.0,0,0,0,...,0,0,0,1,0,0,0,0,0,3


Dividimos la base de datos train en entrenamiento y testeo (Excluimos el rendimiento y el ID)

In [26]:
X = train_listos.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
y = train_listos['RENDIMIENTO_GLOBAL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Nuestro modelo es una regresion logistica

In [28]:
start_time = time.time()

logistic_model = LogisticRegression(
    multi_class='multinomial',
    solver='saga',  
    max_iter=200,   
    n_jobs=-1,      
    random_state=42
)

logistic_model.fit(X_train, y_train)
end_time = time.time()



c:\Users\joseo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Ponemos a predecir en el testeo para evaluar la precision

In [29]:
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Tiempo de entrenamiento: {end_time - start_time} segundos')


Accuracy: 0.3515162454873646
Tiempo de entrenamiento: 2781.3429737091064 segundos


Como al entrenar el modelo quitamos los ID's por no ser significativos tambien se le deben quitar al test para que no arroje errores.

In [30]:
ids = test_listos['ID']

new_data_processed = test_listos.drop('ID', axis=1)

In [31]:
new_data_processed.head()

,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,ESTU_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
692500,20183,1.0,1.0,0.0,1.0,1.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
692501,20203,1.0,1.0,0.0,0.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
692502,20212,1.0,1.0,0.0,0.0,1.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
692503,20195,0.0,1.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
692504,20212,1.0,1.0,0.0,0.0,1.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Predecimos en nuestra base de datos de test y guardamos las predicciones en una nueva columna con los rendimientos predichos

In [32]:

predictions = logistic_model.predict(new_data_processed)

test_listos['RENDIMIENTO_GLOBAL'] = predictions


C:\Users\joseo\AppData\Local\Temp\ipykernel_12856\3774715253.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_listos['RENDIMIENTO_GLOBAL'] = predictions
C:\Users\joseo\AppData\Local\Temp\ipykernel_12856\3774715253.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_listos['RENDIMIENTO_GLOBAL'] = predictions


In [33]:
test_listos.head()

,ID,PERIODO,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,...,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL
692500,550236,20183,1.0,1.0,0.0,1.0,1.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
692501,98545,20203,1.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
692502,499179,20212,1.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
692503,782980,20195,0.0,1.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
692504,785185,20212,1.0,1.0,0.0,0.0,1.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


Preparamos las dos columnas que tendrá la base de datos a subir en Kaggle

In [34]:
test = test_listos[['ID', 'RENDIMIENTO_GLOBAL']]
test.head()

,ID,RENDIMIENTO_GLOBAL
692500,550236,0
692501,98545,0
692502,499179,0
692503,782980,1
692504,785185,1


Reemplazamos las predicciones numericas por la categoria original

In [35]:
mapping = {2: 'medio-alto', 0: 'alto', 1: 'bajo', 3: 'medio-bajo'}

# Aplicar el mapeo a la columna RENDIMIENTO_GLOBAL
test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)

C:\Users\joseo\AppData\Local\Temp\ipykernel_12856\652165175.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)


In [36]:
test.head()

,ID,RENDIMIENTO_GLOBAL
692500,550236,alto
692501,98545,alto
692502,499179,alto
692503,782980,bajo
692504,785185,bajo


Exportamos la base de datos

In [38]:
test.to_csv('test_logistic.csv', index=False)